![](https://www.databricks.com/sites/default/files/inline-images/building-data-pipelines-with-delta-lake-120823.png)

https://www.databricks.com/glossary/medallion-architecture

In [0]:


'''
<storage-account>/<container>/
│
├── projet1/
│   ├── resource/                 # dados de origem e entrega
│   │   ├── origem/               # arquivos CSV/Parquet recebidos da fonte
│   │   └── destino/              # exportações (CSV, Parquet, Excel) para clientes ou setores
│   │
│   ├── bronze/                   # dados crus (raw)
│   │   ├── clientes/
│   │   ├── pedidos/
│   │   ├── produtos/
│   │   └── categorias/
│   │
│   ├── silver/                   # dados limpos / tratados
│   │   ├── clientes/
│   │   └── pedidos/
|   |   └── produtos/
│   │
│   ├── gold/                     # dados agregados / analytics
│   │   └── vendas_categoria/
│   │   └── vendas_cidade_MG/
│   │
│   │
│   ├── metadata/                 # metadados e logs
│   │   ├── bronze/
│   │   ├── silver/
│   │   ├── gold/
│   │   ├── ddl/                  # scripts de criação de tabelas
│   │   ├── logs/                 # logs de execução do ETL
│   │   └── checkpoints/          # checkpoints do Autoloader / streaming
│   │
│   └── tmp/                      # staging temporário

│
├── projet2/
Obs: Estrutura básica de exemplo, cada empresa / setor pode ter a sua 


'''

## Importante
Para Criar tabela com metadados externos a pasta dos metadados "**Não**" podem estar mapeados como "**Volumes**" <br>
Nesse caso, não use Volumes, mas sim um External Location previamente criado e concedido no UC.

In [0]:
%sql
drop schema if exists infinitybigdata.azure cascade

Teste com erro

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS infinitybigdata.azure;
create external volume infinitybigdata.azure.projet1 
comment 'Volume externo na Azure'
location 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/projet1'


In [0]:
%sql
CREATE TABLE infinitybigdata.azure.bronze_clientes
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/projet1/metadados/bronze/custumers'



Metadados fora do volume

In [0]:
%sql
drop volume azure.projet1

In [0]:
%sql
--CREATE SCHEMA IF NOT EXISTS infinitybigdata.azure;
create external volume infinitybigdata.azure.projet1_resource 
comment 'Volume externo na Azure somente recursos e arquivos'
location 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/projet1/resource'


In [0]:
%sql
--Exemplo de tabela vazia só para ver as pastas se comportando 
CREATE TABLE infinitybigdata.azure.bronze_custumers
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/projet1/metadata/bronze/custumers'
-- obs: pasta de Metadados não deve estar mapeadas em Volumes


Formas de Criar e popular tabelas

In [0]:
bronze_path   = 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/projet1/bronze/'
silver_path   = 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/projet1/silver/'
gold_path     = 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/projet1/gold/'
resource_path = 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/projet1/resource/origem/'
resource_path_volume = '/Volumes/infinitybigdata/azure/projet1_resource/origem/'

In [0]:
%fs
ls "abfss://uc-ext-azure@externalazure.dfs.core.windows.net/projet1/resource/origem/"

In [0]:
display(dbutils.fs.ls(resource_path))

In [0]:
display(dbutils.fs.ls(f'{resource_path}'))

In [0]:
# Ler o CSV ou origem qualquer em DataFrame do Spark
df_brands=spark.read.csv(f'{resource_path}/brands.csv',
                          header=True, 
                          inferSchema=True
                          )


In [0]:
display(df_brands)

In [0]:
#salvar em parquet como delta 
df_brands.write\
    .mode('overwrite')\
    .format('delta')\
    .option("mergeSchema", "true") \
    .save(f"{bronze_path}/brands")
#.save(bronze_path+'brands')
''' mergeSchema permite que novas colunas adicionadas ao DataFrame sejam incorporadas ao Delta sem precisar recriar a tabela.
Se você remover uma coluna no DataFrame e fizer um overwrite com mergeSchema=True, a coluna antiga continua existindo na tabela Delta, mas todos os valores correspondentes ficarão nulos.'''

criar tabela sql apartir do arquivo 

In [0]:
display(dbutils.fs.ls(f"{bronze_path}/brands"))

In [0]:
display(dbutils.fs.ls(f"{bronze_path}"))

In [0]:
%sql
--criar uma tabela com o external location
CREATE TABLE IF NOT EXISTS infinitybigdata.azure.bronze_brands
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/projet1/bronze/brands/'

In [0]:
%sql
select * from infinitybigdata.azure.bronze_brands


Modo SQL prático só para teste

In [0]:
%sql
select * from csv.`/Volumes/infinitybigdata/azure/projet1_resource/origem/customers.csv`
WITH (
  header = 'true',
  inferSchema = 'true'
);
--  leitura rapida para ambiente de produção, teste , consulta rapida etc 




In [0]:
%sql
--CTAS create table as + a consulta 
CREATE TABLE IF NOT EXISTS infinitybigdata.azure.bronze_clientes_teste
LOCATION 'abfss://uc-ext-azure@externalazure.dfs.core.windows.net/projet1/metadata/bronze/clientes_teste' 
AS
select * from csv.`/Volumes/infinitybigdata/azure/projet1_resource/origem/customers.csv`
WITH (
  header = 'true',
  inferSchema = 'true'
);


In [0]:
%sql
select * from azure.bronze_clientes_teste
where first_name = 'Jacquline'

In [0]:
%sql
drop volume azure.projet1_resource;
drop schema infinitybigdata.azure cascade